<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# Eikon Data API

**Data for Chains of Options**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial covers

* Retrieving Options Data &mdash; Single Maturity
* Retrieving Options Data &mdash; Multiple Maturities
* Selecting Sub Sets of the Options Data
* Visualization of Sub Sets of the Options Data

## Imports and Versions

The following imports several **packages** as used in the following.

In [1]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline

The following **Python and package versions** are used.

In [2]:
import sys
print(sys.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:49:06) 
[Clang 14.0.6 ]


In [3]:
ek.__version__

'1.1.16'

In [4]:
np.__version__

'1.24.4'

In [5]:
pd.__version__

'2.0.1'

In [6]:
cf.__version__

'0.17.3'

## Connecting to Eikon Data API

This code sets the `app_id` to connect to the **Eikon Data API Proxy** which needs to be running locally. It requires the previously created text file `eikon.cfg` to be in the current working directory.

In [9]:
cfg = cp.ConfigParser()
cfg.read('../refinitiv.cfg')


['../refinitiv.cfg']

In [10]:
ek.set_app_key(cfg['eikon']['app_id']) #set_app_id function being deprecated

## Retrieving Options Data &mdash; Single Maturity

The `RIC` structure provides access to groups of `RICs` that have a common association &mdash; for example, the constituent members of the DAX 30 index or a list of options for a particular underlying contract, such as the DAX 30 index.

For example:

* `0#.GDAX` gives you a list of DAX 30 constituents 

* `0#GDAX*.EX` gives you a list of all options for all maturities for the DAX index
   
* `0#GDAXM8*.EX` gives you a list of all options for June 2018 maturity for the DAX index

### The Raw Data 

In what follows, data is retrieved that comprises fields for the **option type (put or call), the strike price, the closing price and the implied volatility**.

In [11]:
fields = ['PUTCALLIND', 'STRIKE_PRC', 'CF_CLOSE', 'IMP_VOLT']

Using the `ek.get_data()` method, allows data retrieval for a **single maturity of index options and multiple data fields** at the same time.

In [16]:
dax = ek.get_data('0#GDAXM6*.EX', fields=fields)[0]

In [17]:
dax.head()

,Instrument,PUTCALLIND,STRIKE_PRC,CF_CLOSE,IMP_VOLT
0,/.GDAXI,<NA>,<NA>,15901.33,<NA>
1,/GDAX132000F6.EX,CALL,13200,4459.6,23.3886
2,/GDAX132000R6.EX,PUT,13200,537.0,20.6844
3,/GDAX134000F6.EX,CALL,13400,4302.7,22.9334
4,/GDAX134000R6.EX,PUT,13400,564.9,20.278


Let us pick and store the closing value for the index.

In [18]:
GDAXI = dax.iloc[0]['CF_CLOSE']

In [19]:
GDAXI

15901.33

To work with the data, the put options related data rows and call options related data rows are separated into two different `DataFrame` objects.

In [20]:
puts = dax[dax['PUTCALLIND'] == 'PUT ']
calls = dax[dax['PUTCALLIND'] == 'CALL']

In [21]:
puts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, 2 to 70
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  35 non-null     string 
 1   PUTCALLIND  35 non-null     string 
 2   STRIKE_PRC  35 non-null     Int64  
 3   CF_CLOSE    35 non-null     Float64
 4   IMP_VOLT    35 non-null     Float64
dtypes: Float64(2), Int64(1), string(2)
memory usage: 1.7 KB


In [22]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, 1 to 69
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  35 non-null     string 
 1   PUTCALLIND  35 non-null     string 
 2   STRIKE_PRC  35 non-null     Int64  
 3   CF_CLOSE    35 non-null     Float64
 4   IMP_VOLT    35 non-null     Float64
dtypes: Float64(2), Int64(1), string(2)
memory usage: 1.7 KB


It might make sense, to restrict the options data to those options that are not too far in or out of the money.

In [23]:
limit = 2000

In [24]:
puts = puts[abs(puts['STRIKE_PRC'] - GDAXI) < limit]
calls = calls[abs(calls['STRIKE_PRC'] - GDAXI) < limit]

In [25]:
puts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 10 to 48
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  20 non-null     string 
 1   PUTCALLIND  20 non-null     string 
 2   STRIKE_PRC  20 non-null     Int64  
 3   CF_CLOSE    20 non-null     Float64
 4   IMP_VOLT    20 non-null     Float64
dtypes: Float64(2), Int64(1), string(2)
memory usage: 1020.0 bytes


In [26]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 9 to 47
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  20 non-null     string 
 1   PUTCALLIND  20 non-null     string 
 2   STRIKE_PRC  20 non-null     Int64  
 3   CF_CLOSE    20 non-null     Float64
 4   IMP_VOLT    20 non-null     Float64
dtypes: Float64(2), Int64(1), string(2)
memory usage: 1020.0 bytes


### The Data Visualized

The separation allows for easily **visualization** of the closing prices and the implied volatilities.

In [27]:
puts.set_index('STRIKE_PRC')[['CF_CLOSE', 'IMP_VOLT']].iplot(subplots=True,
                                                            mode='lines+markers',
                                                            symbol='circle-dot', size=6)

In [28]:
calls.set_index('STRIKE_PRC')[['CF_CLOSE', 'IMP_VOLT']].iplot(subplots=True,
                                                             mode='lines+markers',
                                                            symbol='circle-dot', size=6)

## Retrieving Data &mdash; Multiple Maturities

Via the `Eikon Data API`, options data can be retrieved also for all maturities of an options chain at once.

### The Raw Data

The maturity date data field (`EXPIR_DATE`) is added to distinguish the maturity dates.

In [29]:
fields = ['PUTCALLIND', 'EXPIR_DATE', 'STRIKE_PRC', 'CF_CLOSE', 'IMP_VOLT']

In [30]:
dax = ek.get_data('0#GDAX*.EX', fields=fields)[0]

In [31]:
dax.head()

,Instrument,PUTCALLIND,EXPIR_DATE,STRIKE_PRC,CF_CLOSE,IMP_VOLT
0,/.GDAXI,<NA>,<NA>,<NA>,15901.33,<NA>
1,/GDAX25000L3.EX,CALL,2023-12-15,2500,13421.7,300.7202
2,/GDAX25000X3.EX,PUT,2023-12-15,2500,0.1,197.5984
3,/GDAX45000L3.EX,CALL,2023-12-15,4500,11427.4,207.8474
4,/GDAX45000X3.EX,PUT,2023-12-15,4500,0.1,135.4583


Again, put and call option data is separated.

In [32]:
puts = dax[dax['PUTCALLIND'] == 'PUT ']
calls = dax[dax['PUTCALLIND'] == 'CALL']

Again, options too far in or out of the money are removed.

In [33]:
puts = puts[abs(puts['STRIKE_PRC'] - GDAXI) < limit]
calls = calls[abs(calls['STRIKE_PRC'] - GDAXI) < limit]

In [34]:
puts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 529 entries, 128 to 2058
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  529 non-null    string 
 1   PUTCALLIND  529 non-null    string 
 2   EXPIR_DATE  529 non-null    string 
 3   STRIKE_PRC  529 non-null    Int64  
 4   CF_CLOSE    518 non-null    Float64
 5   IMP_VOLT    515 non-null    Float64
dtypes: Float64(2), Int64(1), string(3)
memory usage: 30.5 KB


In [35]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 529 entries, 127 to 2057
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instrument  529 non-null    string 
 1   PUTCALLIND  529 non-null    string 
 2   EXPIR_DATE  529 non-null    string 
 3   STRIKE_PRC  529 non-null    Int64  
 4   CF_CLOSE    518 non-null    Float64
 5   IMP_VOLT    518 non-null    Float64
dtypes: Float64(2), Int64(1), string(3)
memory usage: 30.5 KB


From all option maturites ...

In [36]:
exp_dates = sorted(list(set(puts['EXPIR_DATE'])))
exp_dates

['2023-12-15',
 '2024-01-19',
 '2024-02-16',
 '2024-03-15',
 '2024-06-21',
 '2024-09-20',
 '2024-12-20',
 '2025-03-21',
 '2025-06-20',
 '2025-09-19',
 '2025-12-19',
 '2026-03-20',
 '2026-06-19',
 '2026-09-18',
 '2026-12-18',
 '2027-12-17']

... only a sub-set selected.

In [37]:
exp_dates = exp_dates[0:7:3]

In [38]:
exp_dates

['2023-12-15', '2024-03-15', '2024-12-20']

### The Data Visualized

The following visualizes **closing prices and implied volatilities in a single plot per maturity**. First, for the **puts**.

In [39]:
to_plot.iplot?

Object `to_plot.iplot` not found.


In [40]:
for expiry in exp_dates:
    to_plot = puts[puts['EXPIR_DATE'] == expiry]
    to_plot.set_index('STRIKE_PRC')[['CF_CLOSE', 'IMP_VOLT']].iplot(title=expiry,
                                                                    secondary_y='IMP_VOLT',
                                                                    mode='lines+markers',
                                                                    symbol='circle-dot', size=6,
                                                                    xTitle='strike price')

Second, for the **calls**.

In [41]:
for expiry in exp_dates:
    to_plot = calls[calls['EXPIR_DATE'] == expiry]
    to_plot.set_index('STRIKE_PRC')[['CF_CLOSE', 'IMP_VOLT']].iplot(title=expiry,
                                                                   secondary_y='IMP_VOLT',
                                                                   mode='lines+markers',
                                                                   symbol='circle-dot', size=6,
                                                                   xTitle='strike price')

## Comparing Prices & Implied Volatilities

However, often one wishes to compare prices and implied volatilites, respectively, for **different maturities in a single plot**.

### Puts

The code below collects **put option closing prices and implied volatilities in different columns** of two separate `DataFrame` objects.

In [42]:
puts_close = pd.DataFrame()
puts_vols = pd.DataFrame()
for expiry in exp_dates:
    sub_set = puts[puts['EXPIR_DATE'] == expiry]
    puts_close[expiry] = sub_set.set_index('STRIKE_PRC')['CF_CLOSE']
    puts_vols[expiry] = sub_set.set_index('STRIKE_PRC')['IMP_VOLT']

Now, closing prices and implied volatilities can be compared easily in a single plot.

In [43]:
puts_close.dropna().iplot(mode='markers', symbol=['circle-dot', 'cross', 'diamond'], size=6, xTitle='strike price')

In [44]:
puts_vols.dropna().iplot(mode='markers', symbol=['circle-dot', 'cross', 'diamond'], size=6, xTitle='strike price')

### Calls

This code does the same for the **call options**.

In [45]:
calls_close = pd.DataFrame()
calls_vols = pd.DataFrame()
for expiry in exp_dates:
    sub_set = calls[calls['EXPIR_DATE'] == expiry]
    calls_close[expiry] = sub_set.set_index('STRIKE_PRC')['CF_CLOSE']
    calls_vols[expiry] = sub_set.set_index('STRIKE_PRC')['IMP_VOLT']

In [46]:
calls_close.dropna().iplot(mode='markers', symbol=['circle-dot', 'cross', 'diamond'], size=6)

In [47]:
calls_vols.dropna().iplot(mode='markers', symbol=['circle-dot', 'cross', 'diamond'], size=6)

## Conclusions

This tutorial covers:

* Retrieving Options Data &mdash; Single Maturity
* Retrieving Options Data &mdash; Multiple Maturities
* Selecting Sub Sets of the Options Data
* Visualization of Sub Sets of the Options Data

## Eikon Data API Developer Resources

* [Overview](https://developers.thomsonreuters.com/eikon-data-apis) 
* [Quick Start ](https://developers.thomsonreuters.com/eikon-data-apis/quick-start)
* [Documentation](https://developers.thomsonreuters.com/eikon-data-apis/docs)
* [Downloads](https://developers.thomsonreuters.com/eikon-data-apis/downloads)
* [Tutorials](https://developers.thomsonreuters.com/eikon-data-apis/learning)
* [Q&A Forums](https://developers.thomsonreuters.com/eikon-data-apis/qa) 

Data Item Browser Application: Type `DIB` into Eikon Search Bar.

* [Article on Chains](https://developers.thomsonreuters.com/article/simple-chain-objects-ema-part-1)

<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">